In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('entities_clustered_BACKUP.csv')

In [3]:
df.head()

,SOURCEURL,domain,title,entity,label,count,entity_original
0,https://www.energy-storage.news/tesla-lands-ta...,energy-storage.news,Tesla lands tax abatement deal for Texas Megap...,a Memorandum of Understanding,LAW,1,a Memorandum of Understanding
1,https://www.motherjones.com/politics/2025/02/e...,motherjones.com,Everything Elon Musk Touches Is a Conflict of ...,the Clean Air Act,LAW,1,the Clean Air Act
2,https://cyprus-mail.com/2025/04/10/tesla-enter...,cyprus-mail.com,Tesla enters Saudi's nascent EV market | Cypr...,Vision 2030,LAW,1,Vision 2030
3,https://www.domain-b.com/industry/industry-gen...,domain-b.com,Tesla enters Saudi Arabia: A cautious debut am...,Vision 2030,LAW,2,Vision 2030
4,https://www.winnipegfreepress.com/business/202...,winnipegfreepress.com,Tesla launches in Saudi Arabia with its first ...,Vision 2030,LAW,1,Vision 2030


In [4]:
df = df[df['count'] >= 7]

In [5]:
df.head()

,SOURCEURL,domain,title,entity,label,count,entity_original
154,https://www.heritage.org/energy-economics/repo...,heritage.org,After Paris: Next Steps for the Trump Administ...,the Paris Agreement,LAW,19,the Paris Agreement
950,https://paultan.org/2025/03/12/tesla-cybertruc...,paultan.org,Tesla Cybertruck RHD considered – built in Tex...,the Cybertruck Owners Club,PRODUCT,11,Cybertruck
972,https://www.breitbart.com/politics/2025/01/02/...,breitbart.com,Las Vegas Police Thank Elon Musk for Assistanc...,the Cybertruck Owners Club,PRODUCT,10,Cybertruck
977,https://www.torquenews.com/11826/tesla-cybertr...,torquenews.com,A Tesla Cybertruck Owner Says There is a Stran...,the Cybertruck Owners Club,PRODUCT,16,Cybertruck
1001,https://www.indiatoday.in/world/story/united-s...,indiatoday.in,US attacks on New Year: New Orleans Tesla Cybe...,the Cybertruck Owners Club,PRODUCT,8,Cybertruck


In [13]:
df_grouped = df.groupby("SOURCEURL", as_index=False).first()

In [14]:
df_grouped.head()

,SOURCEURL,domain,title,entity,label,count,entity_original
0,http://baystreet.ca/articles/commodities/11182...,baystreet.ca,Baystreet.ca - Taiwan Plans Referendum on Nucl...,Taiwan,GPE,10,Taiwan
1,http://baystreet.ca/articles/commodities/11304...,baystreet.ca,Baystreet.ca - Puerto Rico’s Energy Crisis Dee...,Puerto Rico’s,GPE,7,Puerto Rico
2,http://baystreet.ca/articles/commodities/11316...,baystreet.ca,Baystreet.ca - The Promise and Pitfalls of Mex...,MEXICO CITY,GPE,20,Mexico
3,http://baystreet.ca/articles/commodities/11320...,baystreet.ca,Baystreet.ca - Slovakia Lifts Veto on New EU E...,slovakia,GPE,8,Slovakia
4,http://baystreet.ca/articles/techinsider/10799...,baystreet.ca,Baystreet.ca - Tesla’s Sales Fall 63% In Franc...,tesla,ORG,10,Tesla


In [8]:
!pip install requests beautifulsoup4 tqdm

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import os, re, time, requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd

In [15]:
# === Folder in Google Drive where text files will be saved ===
SAVE_DIR = "/content/drive/MyDrive/NLP project/output_processed/texts"
os.makedirs(SAVE_DIR, exist_ok=True)

In [16]:
# Helper to turn URL into a safe filename
def slugify(url, maxlen=120):
    url = re.sub(r'^https?://', '', url)             # remove http/https
    url = re.sub(r'[^0-9a-zA-Z_-]+', '_', url)       # replace bad chars
    return url[:maxlen].strip('_')

headers = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/124.0 Safari/537.36"
    )
}

# Work on a copy of df_grouped and add the new column
df_grouped = df_grouped.copy()
df_grouped["TEXT_FILE"] = None    # new column for the txt file name

# Loop over each row of the grouped dataframe
for i, (idx, row) in tqdm(enumerate(df_grouped.iterrows()), total=len(df_grouped)):
    url = row["SOURCEURL"]

    # skip if URL is missing
    if pd.isna(url) or not isinstance(url, str):
        continue

    # create a unique filename for this URL
    filename = f"{i:06d}_{slugify(url)}.txt"
    filepath = os.path.join(SAVE_DIR, filename)

    # store the filename in the dataframe
    df_grouped.at[idx, "TEXT_FILE"] = filename

    # if we already downloaded it before, skip downloading
    if os.path.exists(filepath):
        continue

    try:
        # download the page
        resp = requests.get(url, headers=headers, timeout=20)
        resp.raise_for_status()

        # extract visible text only
        soup = BeautifulSoup(resp.text, "html.parser")
        for tag in soup(["script", "style", "noscript"]):
            tag.decompose()
        text = soup.get_text(separator="\n", strip=True)

        # save to .txt file
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(url + "\n\n")   # first line = URL
            f.write(text)

        # be polite to servers
        time.sleep(1)

    except Exception as e:
        print(f"Failed to download {url}: {e}")


  0%|          | 0/24394 [00:00<?, ?it/s]

Failed to download http://businessghana.com/site/news/general/320895/Israel-to-support-Ghana%E2%80%99s-energy-effort: HTTPConnectionPool(host='businessghana.com', port=80): Read timed out. (read timeout=20)
Failed to download http://feeds.theuknews.com/news/278045658/uk-india-convene-4th-energy-dialogue: 403 Client Error: Forbidden for url: http://feeds.theuknews.com/news/278045658/uk-india-convene-4th-energy-dialogue
Failed to download http://www.businessghana.com/site/news/Business/334769/John-Jinapor:-Ghana-to-hit-35-per-cent-renewable-energy-integration-by-December: HTTPConnectionPool(host='www.businessghana.com', port=80): Read timed out. (read timeout=20)
Failed to download http://www.businessghana.com/site/news/business/322096/African-Energy-Week-2025-to-Position-Africa-as-Global-Energy-Champion: HTTPConnectionPool(host='www.businessghana.com', port=80): Read timed out. (read timeout=20)
Failed to download http://www.businessghana.com/site/news/business/325338/Musk-says-Tesla-no

In [18]:
# merge df_grouped (with the TEXT_FILE column) back to the original df
df_ungrouped = df.merge(
    df_grouped[["SOURCEURL", "TEXT_FILE"]],
    on="SOURCEURL",
    how="left"
)

In [19]:
df_ungrouped.to_csv("/content/drive/MyDrive/NLP project/output_processed/df_ungrouped.csv", index=False)


In [21]:
df_ungrouped.head()

,SOURCEURL,domain,title,entity,label,count,entity_original,TEXT_FILE
0,https://www.heritage.org/energy-economics/repo...,heritage.org,After Paris: Next Steps for the Trump Administ...,the Paris Agreement,LAW,19,the Paris Agreement,015524_www_heritage_org_energy-economics_repor...
1,https://paultan.org/2025/03/12/tesla-cybertruc...,paultan.org,Tesla Cybertruck RHD considered – built in Tex...,the Cybertruck Owners Club,PRODUCT,11,Cybertruck,006464_paultan_org_2025_03_12_tesla-cybertruck...
2,https://www.breitbart.com/politics/2025/01/02/...,breitbart.com,Las Vegas Police Thank Elon Musk for Assistanc...,the Cybertruck Owners Club,PRODUCT,10,Cybertruck,009749_www_breitbart_com_politics_2025_01_02_l...
3,https://www.torquenews.com/11826/tesla-cybertr...,torquenews.com,A Tesla Cybertruck Owner Says There is a Stran...,the Cybertruck Owners Club,PRODUCT,16,Cybertruck,023320_www_torquenews_com_11826_tesla-cybertru...
4,https://www.indiatoday.in/world/story/united-s...,indiatoday.in,US attacks on New Year: New Orleans Tesla Cybe...,the Cybertruck Owners Club,PRODUCT,8,Cybertruck,016197_www_indiatoday_in_world_story_united-st...
